In [1]:
%load_ext rustdef

load rustdef


In [2]:
%rustdef depends ndarray

In [3]:
%%rustdef --force-rebuild
use rustdef::numpy::Array;

// immutable example
#[pyfunction]
fn axpy<'a>(a: f64, x: Array<'a, f64>, y: Array<'a, f64>) -> PyResult<Array<'a, f64>> {
    let x = x.to_array();
    let y = y.to_array();
    let z = a * x + y;
    println!("{}", a);
    Ok(z.into())
}

Building..
🔗 Found pyo3 bindings
🐍 Found CPython 3.7m at python3.7
   Compiling rustdef v0.1.0 (/Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/rustdef)
   Compiling rustdef_cell_90ca76e0b1b4a717af6b5fc54e3b7d961fa54e7e v0.1.0 (/Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/rustdef_cell_90ca76e0b1b4a717af6b5fc54e3b7d961fa54e7e)
    Finished release [optimized] target(s) in 4.58s==================> ] 79/80
📦 Built wheel for CPython 3.7m to /Users/ryosuke.kamesawa/Develop/rustdef/examples/.rustdef/target/wheels/rustdef_cell_90ca76e0b1b4a717af6b5fc54e3b7d961fa54e7e-0.1.0-cp37-cp37m-macosx_10_7_x86_64.whl


In [4]:
import numpy as np

In [5]:
axpy(2.4, np.array([1.0, 3.2, 2.9]), np.array([2.3, 10.9, -2]))

2.4


array([ 4.7 , 18.58,  4.96])

In [6]:
%%rustdef
use rustdef::numpy::Array;

// immutable example
#[pyfunction]
fn axpy<'a>(a: f64, x: Array<'a, f64>, y: Array<'a, f64>) -> PyResult<Array<'a, f64>> {
    let x = x.to_array();
    let y = y.to_array();
    let z = a * x + y;
    Ok(z.into())
}

// mutable example
#[pyfunction]
fn mult_inline<'a>(a: f64, x: Array<'a, f64>) -> PyResult<()> {
    let mut x = x.as_mut_array()?;
    x *= a;
    Ok(())
}

Use previous build


In [7]:
x = np.random.rand(10)
y = np.random.rand(10)
x, y

(array([0.19819497, 0.53533892, 0.00374522, 0.8177266 , 0.23339536,
        0.82412973, 0.30025583, 0.54454643, 0.74431044, 0.47844637]),
 array([0.36801197, 0.74525113, 0.94170326, 0.25801958, 0.30415374,
        0.58654776, 0.60195342, 0.13001252, 0.27445097, 0.00601374]))

In [8]:
axpy(3.0, x, y)

array([0.96259688, 2.35126788, 0.95293891, 2.71119937, 1.00433983,
       3.05893695, 1.50272092, 1.76365182, 2.50738229, 1.44135284])

In [9]:
mult_inline(12.0, x)

In [10]:
x

array([2.37833961, 6.42406701, 0.0449426 , 9.81271917, 2.80074434,
       9.88955674, 3.60307002, 6.53455719, 8.93172529, 5.74135641])